<a href="https://colab.research.google.com/github/jun-hyeok/AAI3006-01_Machine-Learning_2021Spring/blob/main/ML_Project_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import numpy as np
import pandas as pd
from google.colab import drive

In [ ]:
drive.mount("/content/drive")
default_dir = "/content/drive/MyDrive/Colab Notebooks/AAI3006-01_Machine-Learning_2021Spring/ML-Project-02/"
df = pd.read_csv(default_dir+"train.csv")
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,emotion_angry_mean,emotion_disgust_mean,emotion_fear_mean,emotion_happy_mean,emotion_sad_mean,emotion_surprise_mean,emotion_neutral_mean,F0semitoneFrom27.5Hz_sma3nz_amean_mean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm_mean,F0semitoneFrom27.5Hz_sma3nz_percentile20.0_mean,F0semitoneFrom27.5Hz_sma3nz_percentile50.0_mean,F0semitoneFrom27.5Hz_sma3nz_percentile80.0_mean,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2_mean,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope_mean,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope_mean,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope_mean,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope_mean,loudness_sma3_amean_mean,loudness_sma3_stddevNorm_mean,loudness_sma3_percentile20.0_mean,loudness_sma3_percentile50.0_mean,loudness_sma3_percentile80.0_mean,loudness_sma3_pctlrange0-2_mean,loudness_sma3_meanRisingSlope_mean,loudness_sma3_stddevRisingSlope_mean,loudness_sma3_meanFallingSlope_mean,loudness_sma3_stddevFallingSlope_mean,spectralFlux_sma3_amean_mean,spectralFlux_sma3_stddevNorm_mean,mfcc1_sma3_amean_mean,mfcc1_sma3_stddevNorm_mean,mfcc2_sma3_amean_mean,mfcc2_sma3_stddevNorm_mean,mfcc3_sma3_amean_mean,mfcc3_sma3_stddevNorm_mean,mfcc4_sma3_amean_mean,mfcc4_sma3_stddevNorm_mean,jitterLocal_sma3nz_amean_mean,jitterLocal_sma3nz_stddevNorm_mean,shimmerLocaldB_sma3nz_amean_mean,...,F2bandwidth_sma3nz_stddevNorm_max,F2amplitudeLogRelF0_sma3nz_amean_max,F2amplitudeLogRelF0_sma3nz_stddevNorm_max,F3frequency_sma3nz_amean_max,F3frequency_sma3nz_stddevNorm_max,F3bandwidth_sma3nz_amean_max,F3bandwidth_sma3nz_stddevNorm_max,F3amplitudeLogRelF0_sma3nz_amean_max,F3amplitudeLogRelF0_sma3nz_stddevNorm_max,alphaRatioV_sma3nz_amean_max,alphaRatioV_sma3nz_stddevNorm_max,hammarbergIndexV_sma3nz_amean_max,hammarbergIndexV_sma3nz_stddevNorm_max,slopeV0-500_sma3nz_amean_max,slopeV0-500_sma3nz_stddevNorm_max,slopeV500-1500_sma3nz_amean_max,slopeV500-1500_sma3nz_stddevNorm_max,spectralFluxV_sma3nz_amean_max,spectralFluxV_sma3nz_stddevNorm_max,mfcc1V_sma3nz_amean_max,mfcc1V_sma3nz_stddevNorm_max,mfcc2V_sma3nz_amean_max,mfcc2V_sma3nz_stddevNorm_max,mfcc3V_sma3nz_amean_max,mfcc3V_sma3nz_stddevNorm_max,mfcc4V_sma3nz_amean_max,mfcc4V_sma3nz_stddevNorm_max,alphaRatioUV_sma3nz_amean_max,hammarbergIndexUV_sma3nz_amean_max,slopeUV0-500_sma3nz_amean_max,slopeUV500-1500_sma3nz_amean_max,spectralFluxUV_sma3nz_amean_max,loudnessPeaksPerSec_max,VoicedSegmentsPerSec_max,MeanVoicedSegmentLengthSec_max,StddevVoicedSegmentLengthSec_max,MeanUnvoicedSegmentLength_max,StddevUnvoicedSegmentLength_max,equivalentSoundLevel_dBp_max,label
0,0.159302,0.001206,0.095556,0.103968,0.241841,0.015937,0.380635,24.365636,0.175620,21.451660,23.824941,26.860989,5.409329,178.302275,118.805772,71.780979,60.595704,1.120236,0.456544,0.679286,1.030189,1.554274,0.874988,11.056342,5.862386,7.338043,4.313646,0.626464,0.695773,21.477432,0.693588,-0.327011,2.948707,2.301753,0.621157,-0.636670,1.357991,0.041140,1.294767,1.411157,...,0.013287,1553.790611,13.057997,17276.461014,0.000760,15273.899997,0.010718,1454.616107,0.333077,11.872866,0.153309,17.167801,0.121265,0.000749,2.103438e+01,0.000062,2.359656,0.082971,0.056775,38.786267,0.067508,39.441458,166082.362816,76.087625,93.992936,77.820486,36532.756730,19.835802,40.867942,0.001236,0.000041,0.073747,3.247842,1.524696,0.038395,0.007741,0.009176,0.002994,17.493596,0
1,0.078909,0.000098,0.097341,0.111471,0.152525,0.038076,0.521042,20.450413,0.180559,17.321590,20.874958,23.396257,6.074668,137.679613,87.802155,67.882583,52.059239,0.622723,0.416345,0.396978,0.559901,0.836382,0.439405,4.619568,2.419374,2.571888,1.489682,0.307432,0.502883,25.386675,0.305130,2.699263,1.668620,4.992532,1.532773,3.519948,2.028082,0.065575,1.384110,1.419527,...,0.005533,491.589328,0.276725,20300.495507,0.000696,10802.614674,0.008120,445.160892,0.125321,4.298156,0.011688,8.891616,0.006371,0.000527,1.106866e+07,0.000032,212.740090,0.012751,0.033280,24.012308,0.017846,21.320284,928.943139,41.933796,520.846650,41.720618,388.881196,15.238695,47.110117,0.000494,0.000053,

In [ ]:
df.shape

(405, 384)

In [ ]:
cols = list(df.columns)
print(len(cols))
print(cols[-1])

384
label


In [ ]:
print(df.dtypes)

emotion_angry_mean                  float64
emotion_disgust_mean                float64
emotion_fear_mean                   float64
emotion_happy_mean                  float64
emotion_sad_mean                    float64
                                     ...   
StddevVoicedSegmentLengthSec_max    float64
MeanUnvoicedSegmentLength_max       float64
StddevUnvoicedSegmentLength_max     float64
equivalentSoundLevel_dBp_max        float64
label                                 int64
Length: 384, dtype: object


In [ ]:
df["label"].values

array([0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,

In [ ]:
import matplotlib.pyplot as plt
cor_mat = df.corr().abs()
cor_mat

,emotion_angry_mean,emotion_disgust_mean,emotion_fear_mean,emotion_happy_mean,emotion_sad_mean,emotion_surprise_mean,emotion_neutral_mean,F0semitoneFrom27.5Hz_sma3nz_amean_mean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm_mean,F0semitoneFrom27.5Hz_sma3nz_percentile20.0_mean,F0semitoneFrom27.5Hz_sma3nz_percentile50.0_mean,F0semitoneFrom27.5Hz_sma3nz_percentile80.0_mean,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2_mean,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope_mean,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope_mean,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope_mean,F0semitoneFrom27.5Hz_sma3nz_stddevFallingSlope_mean,loudness_sma3_amean_mean,loudness_sma3_stddevNorm_mean,loudness_sma3_percentile20.0_mean,loudness_sma3_percentile50.0_mean,loudness_sma3_percentile80.0_mean,loudness_sma3_pctlrange0-2_mean,loudness_sma3_meanRisingSlope_mean,loudness_sma3_stddevRisingSlope_mean,loudness_sma3_meanFallingSlope_mean,loudness_sma3_stddevFallingSlope_mean,spectralFlux_sma3_amean_mean,spectralFlux_sma3_stddevNorm_mean,mfcc1_sma3_amean_mean,mfcc1_sma3_stddevNorm_mean,mfcc2_sma3_amean_mean,mfcc2_sma3_stddevNorm_mean,mfcc3_sma3_amean_mean,mfcc3_sma3_stddevNorm_mean,mfcc4_sma3_amean_mean,mfcc4_sma3_stddevNorm_mean,jitterLocal_sma3nz_amean_mean,jitterLocal_sma3nz_stddevNorm_mean,shimmerLocaldB_sma3nz_amean_mean,...,F2bandwidth_sma3nz_stddevNorm_max,F2amplitudeLogRelF0_sma3nz_amean_max,F2amplitudeLogRelF0_sma3nz_stddevNorm_max,F3frequency_sma3nz_amean_max,F3frequency_sma3nz_stddevNorm_max,F3bandwidth_sma3nz_amean_max,F3bandwidth_sma3nz_stddevNorm_max,F3amplitudeLogRelF0_sma3nz_amean_max,F3amplitudeLogRelF0_sma3nz_stddevNorm_max,alphaRatioV_sma3nz_amean_max,alphaRatioV_sma3nz_stddevNorm_max,hammarbergIndexV_sma3nz_amean_max,hammarbergIndexV_sma3nz_stddevNorm_max,slopeV0-500_sma3nz_amean_max,slopeV0-500_sma3nz_stddevNorm_max,slopeV500-1500_sma3nz_amean_max,slopeV500-1500_sma3nz_stddevNorm_max,spectralFluxV_sma3nz_amean_max,spectralFluxV_sma3nz_stddevNorm_max,mfcc1V_sma3nz_amean_max,mfcc1V_sma3nz_stddevNorm_max,mfcc2V_sma3nz_amean_max,mfcc2V_sma3nz_stddevNorm_max,mfcc3V_sma3nz_amean_max,mfcc3V_sma3nz_stddevNorm_max,mfcc4V_sma3nz_amean_max,mfcc4V_sma3nz_stddevNorm_max,alphaRatioUV_sma3nz_amean_max,hammarbergIndexUV_sma3nz_amean_max,slopeUV0-500_sma3nz_amean_max,slopeUV500-1500_sma3nz_amean_max,spectralFluxUV_sma3nz_amean_max,loudnessPeaksPerSec_max,VoicedSegmentsPerSec_max,MeanVoicedSegmentLengthSec_max,StddevVoicedSegmentLengthSec_max,MeanUnvoicedSegmentLength_max,StddevUnvoicedSegmentLength_max,equivalentSoundLevel_dBp_max,label
emotion_angry_mean,1.000000,0.286104,0.075734,0.346715,0.068168,0.089552,0.212168,0.165754,0.045429,0.116545,0.176296,0.204950,0.114670,0.205992,0.139885,0.028169,0.072243,0.009407,0.118134,0.010966,0.003473,0.017309,0.037549,0.016710,0.003099,0.035496,0.033471,0.069916,0.050995,0.026684,0.013282,0.129820,0.036937,0.092156,0.007111,0.126863,0.114158,0.005151,0.141629,0.112483,...,0.039049,0.007611,0.061736,0.014503,0.025616,0.016859,0.064698,0.004423,0.009902,0.030919,0.064462,0.056899,0.006592,0.139646,0.006466,0.052946,0.049173,0.047031,0.030977,0.034766,0.034173,0.166134,0.020249,0.011948,0.015498,0.100394,0.041877,0.037270,0.052482,0.088982,0.076159,0.056187,0.019282,0.002103,0.026787,0.022276,0.003974,0.025905,0.006409,0.111481
emotion_disgust_mean,0.286104,1.000000,0.064284,0.082997,0.005665,0.062351,0.040413,0.009105,0.050800,0.025027,0.015540,0.004882,0.047662,0.033124,0.030207,0.081453,0.069132,0.020566,0.135328,0.040794,0.027192,0.010306,0.014959,0.001815,0.009409,0.004947,0.003383,0.027754,0.136010,0.035823,0.026338,0.017714,0.038001,0.037094,0.006122,0.081312,0.035744,0.044308,0.016939,0.047882,...,0.004973,0.064345,0.009035,0.020519,0.013585,0.014871,0.004016,0.061545,0.006782,0.011644,0.007375,0.000857,0.008220,0.040167,0.021133,0.025087,0.014760,0.022947,0.001366,0.009389,0.028489,0.005696,0.004458,0.045771,0.003222,0.024156,0.019067,0.049055,0.022580,0.010318,0.022616,0.050902,0.039035,0.074919,0.011940,0.038459,0.03

In [ ]:
mask = np.triu(np.ones(cor_mat.shape), k=1).astype(np.bool)
u_cor_mat = cor_mat.where(mask)
to_drop = [col for col in u_cor_mat.columns if any(u_cor_mat[col] > 0.95)]
df_dropped = df.drop(to_drop, axis=1)
df_dropped.shape

(405, 237)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

## Classifier ##
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

X = df_dropped.drop(["label"], axis=1)
X = X.to_numpy()
y = df_dropped["label"]

X = StandardScaler().fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.4, random_state=42
)

names = ["linear_svm", "kernel_svm", "dt", "rf", "mlp"]
classifiers = [
    SVC(kernel="linear", C=0.025),
    SVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    MLPClassifier(),
]
parameters = {
    "linear_svm": {"kernel": ["linear"], "C": [0.01, 0.5, 1]},
    "kernel_svm": {"kernel": ["rbf", "poly"], "gamma": [1, 2], "C": [1, 2]},
    "dt": {
        "criterion": ["gini", "entropy"],
        "max_depth": [3, 5, 10],
        "min_samples_split": [0.05, 0.1],
    },
    "rf": {
        "criterion": ["gini", "entropy"],
        "max_depth": [3, 5, 10],
        "n_estimators": [100, 1000, 2000],
    },
    "mlp": {
        "hidden_layer_sizes": [100, 500, 1000],
        "activation": ["relu", "logistic", "tanh"],
        "solver": ["adam"],
        "alpha": [0.1, 0.01, 1e-3],
        "batch_size": [10, 30, 100],
        "max_iter": [100, 500, 1000, 2000],
    },
}
model = {}
for name, clf in zip(names, classifiers):
    grid_search = GridSearchCV(clf, parameters[name], n_jobs=-1, cv=3)
    grid_search.fit(X_train, y_train)
    model[name] = {"best_estimator": grid_search.best_estimator_}
    model[name]["score"] = grid_search.best_estimator_.score(X_test, y_test)

In [ ]:
model

{'dt': {'best_estimator': DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                         max_depth=3, max_features=None, max_leaf_nodes=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=0.1,
                         min_weight_fraction_leaf=0.0, presort='deprecated',
                         random_state=None, splitter='best'),
  'score': 0.7037037037037037},
 'kernel_svm': {'best_estimator': SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
      decision_function_shape='ovr', degree=3, gamma=1, kernel='poly',
      max_iter=-1, probability=False, random_state=None, shrinking=True,
      tol=0.001, verbose=False), 'score': 0.7160493827160493},
 'linear_svm': {'best_estimator': SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
      decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
  

In [ ]:
names = ["linear_svm", "kernel_svm", "dt", "rf", "mlp"]
classifiers = [
    SVC(kernel="linear", C=0.025),
    SVC(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    MLPClassifier(),
]
parameters = {
    "linear_svm": {"kernel": ["linear"], "C": [0.01, 0.5, 1]},
    "kernel_svm": {"kernel": ["rbf", "poly"], "gamma": [1, 2], "C": [1, 2]},
    "dt": {
        "criterion": ["gini"],
        "max_depth": [3],
        "min_samples_split": [0.1],
    },
    "rf": {
        "criterion": ["entropy"],
        "max_depth": [5],
        "n_estimators": [100, 1000],
    },
    "mlp": {
        "hidden_layer_sizes": [100],
        "activation": ["logistic"],
        "solver": ["adam"],
        "alpha": [0.01],
        "batch_size": [100],
        "max_iter": [1000],
    },
}
model = {}
for name, clf in zip(names, classifiers):
    grid_search = GridSearchCV(clf, parameters[name], n_jobs=-1, cv=3)
    print(name)
    grid_search.fit(X_train, y_train)
    model[name] = {"best_estimator": grid_search.best_estimator_}
    model[name]["score"] = grid_search.best_estimator_.score(X_test, y_test)

linear_svm
kernel_svm
dt
rf
mlp


In [ ]:
model

{'dt': {'best_estimator': DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                         max_depth=3, max_features=None, max_leaf_nodes=None,
                         min_impurity_decrease=0.0, min_impurity_split=None,
                         min_samples_leaf=1, min_samples_split=0.1,
                         min_weight_fraction_leaf=0.0, presort='deprecated',
                         random_state=None, splitter='best'),
  'score': 0.6975308641975309},
 'kernel_svm': {'best_estimator': SVC(C=1, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
      decision_function_shape='ovr', degree=3, gamma=1, kernel='poly',
      max_iter=-1, probability=False, random_state=None, shrinking=True,
      tol=0.001, verbose=False), 'score': 0.7160493827160493},
 'linear_svm': {'best_estimator': SVC(C=0.01, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
      decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
  

In [ ]:
best_estimator = model[max(model, key=lambda x: model[x]["score"])]["best_estimator"]
print(best_estimator.score(X_test, y_test))

0.7530864197530864


In [ ]:
from sklearn.metrics import classification_report
y_pred = best_estimator.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.76      0.81      0.79        91
           1       0.74      0.68      0.71        71

    accuracy                           0.75       162
   macro avg       0.75      0.74      0.75       162
weighted avg       0.75      0.75      0.75       162



In [ ]:
final = pd.read_csv(default_dir+"X_test.csv").drop(to_drop, axis=1).to_numpy()
pred = best_estimator.predict(final)
pd.DataFrame(pred).to_csv("2018311199_.csv", header=None)

In [ ]:
ada_boost = AdaBoostClassifier(
    base_estimator=model["rf"]["best_estimator"],
    n_estimators=1000,
    learning_rate=0.01,
    random_state=42,
)
ada_boost.fit(X_train, y_train)
print(ada_boost.score(X_test, y_test))

0.7407407407407407


In [ ]:
from sklearn.metrics import classification_report
y_pred = ada_boost.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.78      0.76      0.77        91
           1       0.70      0.72      0.71        71

    accuracy                           0.74       162
   macro avg       0.74      0.74      0.74       162
weighted avg       0.74      0.74      0.74       162



In [ ]:
final = pd.read_csv(default_dir+"X_test.csv").drop(to_drop, axis=1).to_numpy()
pred = ada_boost.predict(final)
pd.DataFrame(pred).to_csv("2018311199.csv", header=None)

In [ ]:
pd.read_csv("2018311199_.csv", index_col=0).to_numpy().reshape(-1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])